Importação das Bibliotecas Necessárias

Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)


Divisão dos Dados em Treino e Teste

In [ ]:
from sklearn.model_selection import train_test_split

# Dividindo os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


Treinamento do Modelo

In [ ]:
from sklearn.linear_model import LogisticRegression

# Instanciando o modelo
model = LogisticRegression(max_iter=1000)

# Treinando o modelo
model.fit(X_train, y_train)


Avaliação do Modelo

Fazendo Previsões

In [ ]:
# Fazendo previsões no conjunto de teste
y_pred = model.predict(X_test)



Acurácia do Modelo

In [ ]:
# Calculando a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")


Análise dos Resultados

Relatório de Classificação

Interpretação:

Precision (Precisão): Proporção de previsões positivas corretas em relação ao total de previsões positivas feitas pelo modelo.
Recall (Sensibilidade): Proporção de casos positivos corretos identificados pelo modelo em relação ao total de casos positivos reais.
F1-Score: Média harmônica entre precisão e recall. É uma métrica balanceada para avaliar o modelo.

In [ ]:
# Exibindo o relatório de classificação
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))


Matriz de Confusão

Interpretação:

Verdadeiros Positivos (TP): Casos em que o modelo previu "Default" e o cliente realmente não pagou.
Verdadeiros Negativos (TN): Casos em que o modelo previu "Reembolso" e o cliente realmente pagou.
Falsos Positivos (FP): Casos em que o modelo previu "Default" mas o cliente pagou.
Falsos Negativos (FN): Casos em que o modelo previu "Reembolso" mas o cliente não pagou.

In [ ]:
# Calculando a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Plotando a matriz de confusão
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Reembolso', 'Default'], yticklabels=['Reembolso', 'Default'])
plt.ylabel('Valor Real')
plt.xlabel('Previsão do Modelo')
plt.title('Matriz de Confusão')
plt.show()


Curva ROC e AUC

Interpretação:

Curva ROC: Mostra a relação entre a taxa de verdadeiros positivos (sensibilidade) e a taxa de falsos positivos para diferentes limiares.
AUC (Area Under the Curve): Mede a capacidade do modelo em classificar corretamente as classes. Valores próximos a 1 indicam um modelo com bom desempenho.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Probabilidades de predição para a classe positiva
y_pred_proba = model.predict_proba(X_test)[:,1]

# Calculando a Curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Calculando a AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Plotando a Curva ROC
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
plt.xlabel('Taxa de Falso Positivo')
plt.ylabel('Taxa de Verdadeiro Positivo')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()


Análise dos Coeficientes do Modelo

Importância das Variáveis

In [ ]:
# Extraindo os coeficientes do modelo
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_[0]
})

# Calculando o valor absoluto dos coeficientes
coefficients['Abs_Coefficient'] = coefficients['Coefficient'].abs()

# Ordenando as variáveis pela importância
coefficients = coefficients.sort_values(by='Abs_Coefficient', ascending=False)

# Visualizando as 10 variáveis mais importantes
print("Coeficientes das Variáveis:")
print(coefficients[['Feature', 'Coefficient']].head(10))


Plotando a Importância das Variáveis

Interpretação:

Os coeficientes positivos indicam que um aumento na variável está associado a um aumento na probabilidade de default.
Os coeficientes negativos indicam que um aumento na variável está associado a uma diminuição na probabilidade de default.
As variáveis com coeficientes de maior magnitude (positiva ou negativa) têm maior impacto nas previsões do modelo.

In [ ]:
# Plotando os coeficientes
plt.figure(figsize=(10,6))
sns.barplot(x='Coefficient', y='Feature', data=coefficients.head(10))
plt.title('Importância das Variáveis (Top 10)')
plt.xlabel('Coeficiente')
plt.ylabel('Variável')
plt.show()


Identificação dos Fatores de Risco
Com base nos coeficientes do modelo, podemos identificar quais variáveis contribuem mais para o risco de crédito.

Análise dos Principais Fatores:
Taxa de Juros do Empréstimo (loan_int_rate):

Coeficiente positivo: Taxas de juros mais altas estão associadas a um maior risco de default.
Interpretação: Clientes com taxas de juros mais altas podem ter maior dificuldade em pagar o empréstimo.
Percentual da Renda Comprometida com o Empréstimo (loan_percent_income):

Coeficiente positivo: Quanto maior o percentual da renda comprometida, maior o risco.
Interpretação: Clientes que comprometem uma grande parte de sua renda com o empréstimo têm maior probabilidade de inadimplência.
Histórico de Crédito (cb_person_cred_hist_length):

Coeficiente negativo: Um histórico de crédito mais longo está associado a um menor risco.
Interpretação: Clientes com um histórico de crédito mais longo geralmente têm mais experiência em lidar com crédito e podem ser considerados menos arriscados.
Intenção do Empréstimo (loan_intent):

Variáveis dummy indicando o propósito do empréstimo (e.g., loan_intent_DEBTCONSOLIDATION).
Certas intenções podem estar associadas a maiores riscos.
Nota de Crédito (loan_grade):

Variáveis dummy para diferentes categorias de nota de crédito.
Notas mais baixas (e.g., loan_grade_G) estão associadas a maior risco.

Visualização da Distribuição das Variáveis
Podemos explorar a distribuição das variáveis mais importantes e como elas diferem entre os clientes que pagaram e os que não pagaram o empréstimo.

Distribuição da Taxa de Juros por Status do Empréstimo

Interpretação:

Clientes que não pagaram tendem a ter taxas de juros mais altas.

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x='loan_status', y='loan_int_rate', data=df_gold)
plt.title('Distribuição da Taxa de Juros por Status do Empréstimo')
plt.xlabel('Status do Empréstimo (0 = Pagou, 1 = Default)')
plt.ylabel('Taxa de Juros (%)')
plt.show()


Distribuição do Percentual de Renda Comprometida

Interpretação:

Clientes inadimplentes geralmente comprometem uma parcela maior de sua renda com o empréstimo.

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x='loan_status', y='loan_percent_income', data=df_gold)
plt.title('Percentual de Renda Comprometida por Status do Empréstimo')
plt.xlabel('Status do Empréstimo (0 = Pagou, 1 = Default)')
plt.ylabel('Percentual de Renda Comprometida')
plt.show()


Conclusões e Insights
Desempenho do Modelo:

Acurácia: O modelo possui uma acurácia de aproximadamente X% (substitua pelo valor real obtido).
AUC: A área sob a curva ROC é de Y (substitua pelo valor real), indicando que o modelo tem um bom poder discriminativo.
Principais Fatores de Risco:

Taxa de Juros Elevada: Clientes com taxas de juros mais altas têm maior risco de default.
Alto Comprometimento da Renda: Um alto percentual da renda comprometida com o empréstimo aumenta o risco.
Curto Histórico de Crédito: Clientes com um histórico de crédito mais curto estão associados a um risco maior.
Intenção do Empréstimo: Certos propósitos do empréstimo, como consolidação de dívidas, podem estar associados a maiores riscos.
Recomendações:

Avaliação Rigorosa: Realizar uma análise mais detalhada para clientes com taxas de juros altas e alto comprometimento de renda.
Políticas de Crédito: Revisar as políticas para aprovar empréstimos com base nos fatores de risco identificados.
Educação Financeira: Oferecer programas de educação financeira para clientes com histórico de crédito curto.

In [7]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Definição do Diretório Base e Criação das Pastas

In [8]:
# Diretório base para as camadas do ETL
base_dir = '/home/savidotti/programmers/credit-risk-analysis/pipeline_etl/etl_camadas'

# Função para criar o diretório se não existir
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

Camada Bronze: Carregamento dos Dados Brutos

In [9]:
# Caminho para o arquivo CSV original
raw_data_path = '/home/savidotti/programmers/credit-risk-analysis/dataset/credit_risk_dataset.csv'

# Carregando os dados brutos
df_bronze = pd.read_csv(raw_data_path)

# Visualizando as primeiras linhas do DataFrame
print("Dados da Camada Bronze:")
print(df_bronze.head())

# Criando o diretório para a camada bronze
bronze_dir = os.path.join(base_dir, 'bronze')
create_dir(bronze_dir)

# Salvando o DataFrame da camada bronze em formato CSV
bronze_file_path = os.path.join(bronze_dir, 'credit_risk_bronze.csv')
df_bronze.to_csv(bronze_file_path, index=False)


Dados da Camada Bronze:
   person_age  person_income person_home_ownership  person_emp_length  \
0          22          59000                  RENT              123.0   
1          21           9600                   OWN                5.0   
2          25           9600              MORTGAGE                1.0   
3          23          65500                  RENT                4.0   
4          24          54400                  RENT                8.0   

  loan_intent loan_grade  loan_amnt  loan_int_rate  loan_status  \
0    PERSONAL          D      35000          16.02            1   
1   EDUCATION          B       1000          11.14            0   
2     MEDICAL          C       5500          12.87            1   
3     MEDICAL          C      35000          15.23            1   
4     MEDICAL          C      35000          14.27            1   

   loan_percent_income cb_person_default_on_file  cb_person_cred_hist_length  
0                 0.59                         Y       

Camada Silver: Limpeza e Transformação Básica

In [10]:
# Criando uma cópia dos dados brutos
df_silver = df_bronze.copy()

# Verificando valores ausentes
print("\nValores ausentes antes do tratamento:")
print(df_silver.isnull().sum())

# Tratamento de valores ausentes (se houver)
if df_silver['person_emp_length'].isnull().sum() > 0:
    df_silver['person_emp_length'].fillna(df_silver['person_emp_length'].median(), inplace=True)

# Convertendo tipos de dados
df_silver['person_age'] = df_silver['person_age'].astype(int)
df_silver['person_income'] = df_silver['person_income'].astype(float)
df_silver['loan_amnt'] = df_silver['loan_amnt'].astype(float)
df_silver['loan_int_rate'] = df_silver['loan_int_rate'].astype(float)
df_silver['loan_status'] = df_silver['loan_status'].astype(int)
df_silver['loan_percent_income'] = df_silver['loan_percent_income'].astype(float)
df_silver['cb_person_cred_hist_length'] = df_silver['cb_person_cred_hist_length'].astype(int)

# Verificando valores ausentes após o tratamento
print("\nValores ausentes após o tratamento:")
print(df_silver.isnull().sum())

# Criando o diretório para a camada silver
silver_dir = os.path.join(base_dir, 'silver')
create_dir(silver_dir)

# Salvando o DataFrame da camada silver em formato Parquet
silver_file_path = os.path.join(silver_dir, 'credit_risk_silver.parquet')
df_silver.to_parquet(silver_file_path, index=False)



Valores ausentes antes do tratamento:
person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64

Valores ausentes após o tratamento:
person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length                0
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64


Camada Gold: Enriquecimento e Preparação para ML

In [12]:
# Criando uma cópia dos dados da camada silver
df_gold = df_silver.copy()

# Engenharia de Características

# Criar uma variável binária para indicar se a pessoa possui casa própria
df_gold['is_home_owner'] = df_gold['person_home_ownership'].apply(lambda x: 1 if x in ['OWN', 'MORTGAGE'] else 0)

# Tratamento de valores nulos em 'person_home_ownership' (se houver)
if df_gold['person_home_ownership'].isnull().sum() > 0:
    df_gold['person_home_ownership'].fillna('OTHER', inplace=True)

# Codificação de variáveis categóricas
categorical_vars = ['loan_intent', 'loan_grade', 'cb_person_default_on_file']

# Usando one-hot encoding para variáveis categóricas
df_gold = pd.get_dummies(df_gold, columns=categorical_vars, drop_first=True)

# Atualizando a lista de features com as novas colunas criadas
features = [
    'person_age',
    'person_income',
    'person_emp_length',
    'loan_amnt',
    'loan_int_rate',
    'loan_percent_income',
    'cb_person_cred_hist_length',
    'is_home_owner'
]

# Adicionando as colunas de variáveis dummies
features += [
    col for col in df_gold.columns 
    if col.startswith('loan_intent_') 
    or col.startswith('loan_grade_') 
    or col.startswith('cb_person_default_on_file_')
]

# Definindo as variáveis X e y
X = df_gold[features]
y = df_gold['loan_status']

# Criando o diretório para a camada gold
gold_dir = os.path.join(base_dir, 'gold')
create_dir(gold_dir)

# Salvando o DataFrame da camada gold em formato Parquet
gold_file_path = os.path.join(gold_dir, 'credit_risk_gold.parquet')
df_gold.to_parquet(gold_file_path, index=False)
